In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as ex
import bz2
import torch
import _pickle as pickle
import scipy.stats as stats
import scipy
from plotly.subplots import make_subplots
import plotly.io as pio
import sys, os

# Functions and global variables

In [2]:
RESULT_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results"
REF_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REF"
TRANS_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/TRANS"
REDC_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REDUC"

METRICS = ["recon_error","adv_error", "rot_error", "delta_rot"]

In [3]:
def load(file_path:str):
    with bz2.BZ2File(file_path, "rb") as f:
        obj = pickle.load(f)
    return obj
def save(file:object, file_path:str):
    with bz2.BZ2File(file_path, "w") as f:
        pickle.dump(file, f)
def calc_RMS(delta_rot:list, dry_run=False):
    rms = lambda x: np.sqrt(np.mean(x**2))
    for i in range(len(delta_rot)):
        rms_x = rms(np.asarray(delta_rot[i][0], np.float64))
        rms_y = rms(np.asarray(delta_rot[i][1], np.float64))
        if not dry_run:
            delta_rot[i] = rms_x+rms_y
def extract_model_results(model_name:str, results:list, dry_run=False):
    df = {}
    length = 0
    for metric in METRICS:
        if length == 0: length = len(results[0][metric])
        df[metric] = [re[metric] for re in results]
        if metric == "delta_rot":
            for delta_rot in df[metric]:
                calc_RMS(delta_rot, dry_run)

    df["name"] = [model_name] * length
    return df

# Prepare data
## Reference data

In [30]:
ref_results = load(os.path.join(RESULT_PATH,"reference_results.pbz2"))

In [35]:
models = list(ref_results[0].keys())
for m in models:
    df = extract_model_results(m, [re[m] for re in ref_results])
    save(df, os.path.join(REF_SAVE_PATH, m+".pbz2"))


IndexError: invalid index to scalar variable.

In [24]:
# REF REDUC
ref_reduc_results = load(os.path.join(RESULT_PATH, "reduc_dfs.pbz2"))

In [25]:
models = list(ref_reduc_results[0].keys())
print(models)
print(ref_reduc_results[0][models[0]].keys())
print(ref_reduc_results[0][models[0]]["sum_delta_rot_x"])
# for m in models:
#     df = extract_model_results(m, [re[m] for re in ref_reduc_results])
#     # save(df, os.path.join(REF_SAVE_PATH, m+".pbz2"))

['AE+MoE_T', 'AE+MoE_F', 'DEC-CAT+MoE_F', 'DEC-CAT+MoE_T', 'RBF-CAT+MoE_F', 'RBF-CAT+MoE_T', 'VAE-CAT+MoE_F', 'VAE-CAT+MoE_T']
Index(['recon_err', 'adv_err', 'rot_err', 'sum_delta_rot_x', 'sum_delta_rot_y',
       'avg_sum_pCost_x', 'avg_sum_pCost_y', 'avg_sum_rCost_x',
       'avg_sum_rCost_t'],
      dtype='object')
0      0       9.051269
1      12.623844
2       6.83...
1                                                    NaN
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
802                                                  NaN
803                                                  NaN
804                                                  NaN
805                                                  NaN
806                                                  NaN
Name: sum_delta_rot_x, Length: 807, dtype: object


## Tranferred results

In [47]:
trans_results = load(os.path.join(RESULT_PATH,"transfer_results.pbz2")) 

In [48]:
models = list(trans_results[0].keys())
for m in models:
    if "LSTM" in m: continue
    df = extract_model_results(m, [re[m] for re in trans_results], dry_run=False)
    save(df, os.path.join(TRANS_SAVE_PATH, m+".pbz2"))


## Transferred results - Reduced

In [49]:
reduc_results = load(os.path.join(RESULT_PATH,"transfer_results_reduc.pbz2")) 

In [50]:
models = list(reduc_results[0].keys())
for m in models:
    if "LSTM" in m: continue
    df = extract_model_results(m, [re[m] for re in reduc_results], dry_run=False)
    save(df, os.path.join(REDC_SAVE_PATH, m+".pbz2"))